<a href="https://colab.research.google.com/github/zemersonnn/Home_Sales/blob/main/Home_Sales_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

In [3]:
import os

In [4]:
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [5]:
!pip install findspark

In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=744bf3ea0975a098c142e8b8f13074e8960eebd6591201cd795979a0022b6177
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [7]:
! pip install py4j

In [8]:
! pip uninstall py4j

Found existing installation: py4j 0.10.9.7
Uninstalling py4j-0.10.9.7:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/py4j-0.10.9.7.dist-info/*
    /usr/local/lib/python3.10/dist-packages/py4j/*
    /usr/local/share/py4j/py4j0.10.9.7.jar
Proceed (Y/n)? n


In [10]:
! pip install py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 1.7 MB/s eta 0:00:00


In [9]:
import os

# Get the current user's home directory
home_directory = os.path.expanduser("~")

# Set the SPARK_HOME environment variable
os.environ["SPARK_HOME"] = f"{home_directory}/spark-3.0.1-bin-hadoop3.2"

In [10]:
# Print the value of the SPARK_HOME environment variable
print(os.environ["SPARK_HOME"])

/root/spark-3.0.1-bin-hadoop3.2


In [11]:
# Restart the Python interpreter to ensure the environment variable is updated
exit()

In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_data')
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import mean
from pyspark.sql.functions import round, col

four_bedroom = df.filter(df.bedrooms == 4)\
  .groupBy("date")\
  .agg(round(mean("price"),2).alias("avg_price"))

four_bedroom.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-11-13| 358445.0|
|2019-05-08| 234979.5|
|2021-08-27|275547.25|
|2021-01-27|367520.57|
|2021-06-22| 297708.8|
|2021-10-11|283388.33|
|2020-08-24|301080.75|
|2022-03-28|338544.78|
|2019-06-04|306373.08|
|2021-12-18| 305264.3|
|2020-11-29|318153.89|
|2020-01-21| 420576.0|
|2020-07-24| 293661.5|
|2021-11-25|291762.27|
|2019-11-18|290259.83|
|2019-09-22| 284864.5|
|2020-08-05|243835.56|
|2019-11-01|254751.38|
|2021-10-02|303838.38|
|2020-08-29| 285457.3|
+----------+---------+
only showing top 20 rows



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bedroom = df.filter((df.bedrooms == 3)&(df.bathrooms == 3))\
  .groupBy("date")\
  .agg(round(mean("price"), 2).alias("avg_price"))

three_bedroom.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-05-08|285754.71|
|2021-06-22| 306565.0|
|2021-01-27|385711.75|
|2020-08-24|319252.33|
|2021-11-13|348180.33|
|2022-03-28| 280464.4|
|2021-10-11| 289886.4|
|2021-12-18| 178508.5|
|2021-08-27|298209.33|
|2019-06-04| 223109.0|
|2020-08-05|304566.75|
|2019-11-18|319833.67|
|2020-01-21| 303045.8|
|2021-11-25|321816.67|
|2019-11-01| 279354.0|
|2020-07-24| 283740.0|
|2020-11-29| 366673.0|
|2021-10-02| 400307.5|
|2019-09-22| 350352.5|
|2020-04-30| 192065.5|
+----------+---------+
only showing top 20 rows



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
three_bedroom_sq_foot = df.filter((df.bedrooms == 3)&(df.bathrooms == 3)&(df.floors == 2)&(df.sqft_living >= 2000))\
  .groupBy("date")\
  .agg(round(mean("price"), 2).alias("avg_price"))

three_bedroom_sq_foot.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-06-22| 420422.0|
|2021-01-27| 446538.0|
|2021-12-18| 178508.5|
|2021-11-13| 697736.0|
|2021-08-27| 385686.0|
|2022-03-28| 204818.0|
|2021-10-11| 167892.0|
|2019-05-08| 391556.0|
|2019-11-01|201940.33|
|2019-11-18| 398256.0|
|2021-10-02| 402262.0|
|2019-09-22| 365867.0|
|2020-01-21| 293205.0|
|2020-04-30| 232217.0|
|2022-01-31| 284811.0|
|2022-05-19|302160.33|
|2020-08-29| 423501.0|
|2020-08-28| 296028.5|
|2019-11-21| 428167.0|
|2021-07-30| 346839.0|
+----------+---------+
only showing top 20 rows



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00012111663818359375 seconds ---


In [14]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.



avg_price = df.filter(df.price >= 350000)\
  .select(mean("price").alias("avg_price"))\
  .collect()[0]["avg_price"]
view_rating = None
if avg_price < 500000:
  view_rating = "Low"
elif avg_price >= 500000 and avg_price < 800000:
  view_rating = "Medium"
else:
  view_rating = "High"
  print("Average Price:", round(avg_price, 2))
print("View Rating:", view_rating)



View Rating: Low


In [16]:
# 7. Cache the the temporary table home_data
spark.catalog.cacheTable("home_data")

In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_data')

True

In [19]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 8.869171142578125e-05 seconds ---


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('p_sales', mode='overwrite')

In [21]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet('p_sales')

In [22]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('p_sales')

In [23]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 9.870529174804688e-05 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_data")

In [25]:
# 15. Check if the home_sales is no longer cached
spark.stop()

In [26]:
spark.catalog.isCached('home_data')

False